# predict .py

In [117]:
from createdataset import *
Label_file, Input_file = ChooseDataset('dev', ['pku'])

In [118]:
Label_file

'../icwb2-data/gold/pku_test_gold_simplified_LabelFile.utf8'

In [119]:
Input_file

'../icwb2-data/gold/pku_test_gold_simplified_InputFile.utf8'

In [124]:
line = 32

In [125]:
f = open('preds.txt').readlines()
len(f[line])

14

In [126]:
f1 = open(Label_file).readlines()
len(f1[line])

60

In [127]:
f2 = open(Input_file).readlines()
len(f2[line])

60

In [ ]:
[len(l) for l in f2][-10:]

In [ ]:
[len(l) for l in f1][-10:]

In [168]:
import json
import numpy as np
import os
import tensorflow.keras as K

from createdataset import *

In [169]:
os.listdir("../resources/models")

['model_2019-04-24_00:47:52_-0500cityu_pku.h5',
 '.DS_Store',
 'model_2019-04-24_00:05:02_-0500msr_pku.h5',
 'model_2019-04-23_23:39:49_-0500cityu.h5',
 'model_weights_2019-04-23_23:39:49_-0500cityu.h5',
 'model_weights_2019-04-24_00:47:52_-0500cityu_pku.h5',
 'model_2019-04-24_06_12_52_+0000cityu.h5',
 'model_weights_2019-04-24_00:05:02_-0500msr_pku.h5',
 'model_weights_2019-04-24_06_12_52_+0000cityu.h5']

In [172]:
input_path = '../icwb2-data/gold/msr_test_gold_simplified_InputFile.utf8'
output_path = 'preds.txt'
resources_path = '../resources'

In [173]:
with open(os.path.join(resources_path, 'vocabs/cityu_30.json')) as f:
    loaded_data = json.load(f)
# processing the Input file into feature vectors given the dictionary
A = CreateDataset(LabelFile_path = None,
                  InputFile_path = input_path,
                  PaddingSize = None,
                  set_type = 'TEST',
                  TrainingVocab = [loaded_data['UNIGRAM_DICT'],
                                   loaded_data['BIGRAM_DICT']])
X_test_uni, X_test_bi = A.DateGenTest()
PAD = 1 #UNK
padded = lambda x: np.array([xi+[PAD]*(max(map(len, x))-len(xi)) for xi in x])
#keeping original lengths
lengths = [len(i) for i in X_test_uni]
#pad to max line length

##################
# Model loading #
#################
loaded_model = K.models.load_model(os.path.join(resources_path, 'models/model_2019-04-24_06_12_52_+0000cityu.h5'))
loaded_model.load_weights(os.path.join(resources_path, 'models/model_weights_2019-04-24_06_12_52_+0000cityu.h5'))
loaded_model.compile(loss = 'categorical_crossentropy',
              optimizer = K.optimizers.Adam(lr=0.0015, clipnorm=1., clipvalue=0.5),
              metrics = ['acc'])

## PREDICTION
predicted_one_hot = loaded_model.predict([padded(X_test_uni), padded(X_test_bi)])
#de - one hot encode
predicted = np.argmax(predicted_one_hot, axis=2)

predictedBIES = []
BIES = {0 : 'B', 1 : 'I', 2 : 'E', 3 : 'S'}
for i in range(len(predicted)):
    #un-padding to original line lengths
    to_bies = list(predicted[i][:lengths[i]])
    #decoding into BIES format from numerical classes for each line
    BIES_letters = ''.join([BIES[i] for i in to_bies])
    #appending to list of lines in BIES
    predictedBIES.append(BIES_letters)

In [174]:
with open(output_path, 'w') as outfile:
    outfile.write('\n'.join(predictedBIES))
    print("Saved predictions at: {}".format(output_path))

Saved predictions at: preds.txt


In [175]:
def label_text_to_iter(file_path):
    iter_ = []
    with open(file_path) as f:
        for line in f:
            line = line.strip().upper()
            iter_.append(line)
    return iter_

In [176]:
pred = label_text_to_iter("preds.txt")

In [177]:
gold = label_text_to_iter("../icwb2-data/gold/msr_test_gold_simplified_LabelFile.utf8")

In [178]:
for i in range(len(pred)):
    if not len(pred[i]) == len(gold[i]):
        print(i)
        print(pred[i])
        print(gold[i])
        
    

In [148]:
len(pred[7768]), len(gold[7768])

IndexError: list index out of range

In [90]:
pred[7768]

''

In [91]:
gold[7768]

'S'

In [92]:
len(X_test_uni[7768])

0

In [69]:
def split_into_grams(sentence: str, type_ = 'uni_grams') -> List[str]:
        """
        :param sentence Sentence as str
        :type_: uni_grams or _bigrams
        :return bigrams List of unigrams or bigrams
        """
        n = 1 if type_ == 'uni_grams' else 2
        grams = []
        for i in range(len(sentence)): #not -1
            gram = sentence[i:i+n]
            grams.append(gram)
        return grams


In [78]:
f = label_text_to_iter("../icwb2-data/gold/as_testing_gold_simplified.utf8")

In [93]:
f[7768]

''

In [94]:
len(split_into_grams(f[7768]))

0

In [95]:
split_into_grams(f[7768], 'bigrams')

[]

In [101]:
def BIESencoder(line, type_of_file):
    '''encodes a single line to the BIES Format'''

    new_line = ''
    #uses the +u3000 seperator if the file is 'AS'
    words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
    #print("N of segments:",len(words))
    def bies_format(word):
        '''changes word to BIES format'''
        if len(word)>1:
            return 'B'+'I'*int(len(word)-2)+'E'
        elif len(word) == 1:
            return 'S'
        else:
            return ''

    for word in words:
        #if verbose: print("this is a word: {} of length {}".format(word, len(word)))
        new_line+=bies_format(word)

    return new_line

In [102]:
BIESencoder(f[7768], 'as')

''

In [97]:
f[7768]

''